### 1. Simplest Deep Neural Network

In [1]:
import torch
import torch.nn as nn

class DNN1(nn.Module):
    def __init__(self, input_size: int, num_labels: int):
        super().__init__()
        self.classifier = nn.Sequential(
                                        nn.Linear(input_size, 1024), nn.ReLU(True),
                                        nn.Linear(1024, 1024), nn.ReLU(True),
                                        nn.Linear(1024, num_labels),
                                        )
    def forward(self, input):
        outputs = self.classifier(input)
        return outputs

In [2]:
import pandas as pd
datapath = "/data/lujd/algorithm2022/audioset/"
dev_df = pd.read_csv(datapath+"dev.csv", sep="\t")

# dev set
import numpy as np
from Extract_feature import extract_mfcc

n_mels = 26
feature_option = "mfcc"
dev_files = list(dev_df.filename)
dev_labels = list(dev_df.scene_label)
dev_features = []
for filename in dev_files:
    wav_file_path = datapath+filename
    mfcc_feature = extract_mfcc(wav_file_path, n_mels=n_mels, option=feature_option)
    dev_features.append(mfcc_feature.reshape(1,-1))             # [1, frames*n_features] (flatten)
dev_features = np.concatenate(dev_features, axis=0)
print(f"dev feature size: {dev_features.shape}")

dev feature size: (379, 6487)


In [3]:
unique_labels = ['airport', 'bus', 'metro', 'metro_station',
                'park', 'public_square', 'shopping_mall',
                'street_pedestrian', 'street_traffic', 'tram']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = DNN1(dev_features.shape[1], len(unique_labels)).to(device)
model.eval()

batch_size = 128
start_pos, end_pos = 0, 0
while end_pos < len(dev_features):
    end_pos = end_pos+batch_size if (end_pos+batch_size)<len(dev_features) else len(dev_features)
    input = torch.Tensor(dev_features[start_pos:end_pos]).to(device)
    print(start_pos, end_pos)
    output = model(input)
    start_pos = end_pos
    print(f"output size: {output.shape}")

0 128
output size: torch.Size([128, 10])
128 256
output size: torch.Size([128, 10])
256 379
output size: torch.Size([123, 10])


In [4]:
output[:3]

tensor([[-0.1643,  0.3229, -0.0079, -0.1673, -0.0643, -0.1362,  0.0104, -0.0590,
         -0.0394,  0.2377],
        [-0.0589,  0.1437, -0.0288, -0.1114, -0.0114, -0.0361, -0.0391, -0.0539,
         -0.0017,  0.1393],
        [-0.0793,  0.1457, -0.0448, -0.1063,  0.0037, -0.0875, -0.0102, -0.0737,
         -0.0023,  0.1311]], device='cuda:0', grad_fn=<SliceBackward>)